# 🎓 AURA V9 CERBERUS - Study Edition (Deep Dive)

Questa edizione contiene il codice completo del protocollo **AURA V9 CERBERUS**, arricchito con spiegazioni teoriche avanzate.
È pensato per essere letto come un libro di testo interattivo.

---

## 🧠 Teoria Avanzata: Perché CERBERUS Funziona?

### 1. Hard Parameter Sharing (L'Architettura)
Non stiamo addestrando 3 modelli separati. Stiamo addestrando un **unico cervello centrale (Encoder)** che condivide i suoi pesi ('hard sharing') per risolvere 3 problemi diversi. 
- **Vantaggio**: Il cervello è costretto a trovare pattern universali. Non può "barare" memorizzando scorciatoie per un solo task.
- **Effetto**: Questo funge da potente regolarizzatore, riducendo drasticamente l'overfitting.

### 2. Homoscedastic Uncertainty (La Matematica)
Usiamo la **Kendall Loss** (`1/σ² * L + log(σ²)`) per bilanciare i task.
- Non è un'incertezza sui dati (Aleatoric Heteroscedastic), ma un'incertezza sul **COMPITO** (Aleatoric Homoscedastic).
- Il modello impara un parametro $\sigma$ fisso per ogni task che rappresenta quanto quel compito sia "rumoroso" o difficile in generale.
- **Risultato**: Il modello impara da solo a ignorare i task troppo rumorosi nelle prime fasi, per concentrarsi su quelli puliti, e poi integrare i difficili gradualmente.

### 3. Differential Learning Rates (L'Ottimizzazione)
Trattiamo gli strati della rete in modo gerarchico:
- **BERT (Il Saggio)**: Ha già letto tutta Wikipedia. Usiamo un LR basso (`2e-5`) per non distruggere la sua conoscenza (evitando il *Catastrophic Forgetting*).
- **Teste (Gli Apprendisti)**: Sono inizializzate a caso. Usiamo un LR alto (`5e-5`) perché devono imparare tutto da zero rapidamente.

---

In [ ]:
# 1. SETUP E RIPRODUCIBILITÀ

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch.optim.lr_scheduler import OneCycleLR
from transformers import BertModel, BertTokenizer
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# DETERMINISMO SCIENTIFICO
# Per pubblicare un paper, i risultati devono essere riproducibili.
# Impostando questi seed, blocchiamo il caos: la sequenza di numeri casuali sarà identica a ogni run.
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True  # Disabilita algoritmi di convoluzione non deterministici sulla GPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device attivo: {device}')

In [ ]:
# 2. HYPERPARAMETERS

CONFIG = {
    'encoder': 'bert-base-uncased',
    'max_length': 128,
    'num_emotion_classes': 7,
    'dropout': 0.2,
    'batch_size': 32,
    
    # GRADIENT ACCUMULATION
    # Permette di simulare un batch size enorme (es. 64 o 128) anche con poca VRAM.
    # Invece di fare 'step()' ogni 32 esempi, accumuliamo gli errori per 2 cicli (32*2 = 64 esempi)
    # e poi facciamo un solo passo di aggiornamento preciso.
    'gradient_accumulation': 2,    

    'epochs': 4,
    'lr_bert': 2e-5,    # Basso per il backbone
    'lr_heads': 5e-5,   # Alto per i classificatori
    'weight_decay': 0.01,
    'warmup_ratio': 0.1,
    'patience': 3,
    'focal_gamma': 2.0,
}

EMO_COLS = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
DATA_DIR = '/kaggle/input/aura-v9-data' 
print("Configurazione pronta.")

In [ ]:
# 3. ARCHITETTURA (Hard Parameter Sharing)

class AURA_CERBERUS(nn.Module):
    def __init__(self, config):
        super().__init__()
        # ENCODER CONDIVISO
        # Questo è il collo di bottiglia dell'informazione.
        # Tutto (Tossicità, Emozioni, Sentiment) deve passare da qui.
        self.bert = BertModel.from_pretrained(config['encoder'])
        hidden = self.bert.config.hidden_size
        self.dropout = nn.Dropout(config['dropout'])
        
        # TASK-SPECIFIC HEADS
        # Layer leggeri che 'interpetano' il pensiero di BERT per scopi diversi.
        self.toxicity_head = nn.Linear(hidden, 2)
        self.emotion_head = nn.Linear(hidden, config['num_emotion_classes'])
        self.sentiment_head = nn.Linear(hidden, 2)
        
        # PARAMETRI INCERTEZZA (Homoscedastic Aleatoric Uncertainty)
        # Inizializzati come scalari logaritmici.
        # log_var = 0.0 corrisponde a sigma = exp(0) = 1.0 (incertezza neutra).
        # Se diventano positivi -> incertezza sale -> loss viene penalizzata.
        # Se diventano negativi -> incertezza scende -> loss viene amplificata.
        self.log_var_tox = nn.Parameter(torch.tensor(0.0))
        self.log_var_emo = nn.Parameter(torch.tensor(0.0))
        self.log_var_sent = nn.Parameter(torch.tensor(0.0))
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = self.dropout(outputs.pooler_output)
        
        return {
            'toxicity': self.toxicity_head(pooled),
            'emotion': self.emotion_head(pooled),
            'sentiment': self.sentiment_head(pooled),
            'log_var_tox': self.log_var_tox,
            'log_var_emo': self.log_var_emo,
            'log_var_sent': self.log_var_sent
        }

In [ ]:
# 4. MATEMATICA E LOSS FUNCTIONS

# FOCAL LOSS (Lin et al., 2017)
# Risolve lo sbilanciamento delle classi in modo dinamico.
# Invece di pesare le classi a priori (es: 'Tossico vale doppio'), pesa gli ESEMPI.
# - Esempio Facile (p=0.9): Focal Loss ≈ 0. Non sprecare gradienti qui.
# - Esempio Difficile (p=0.4): Focal Loss Alta. Concentrati qui.
def focal_loss(logits, targets, gamma=2.0, weight=None):
    ce = F.cross_entropy(logits, targets, weight=weight, reduction='none')
    p_t = torch.exp(-ce)
    loss = ((1 - p_t) ** gamma * ce).mean()
    return loss

# KENDALL LOSS (Kendall et al., 2018)
# La formula magica per Multi-Task Learning.
# Loss = (TaskLoss / 2σ²) + log(σ)
# - Il primo termine premia la precisione ma divide per l'incertezza.
# - Il secondo termine impedisce al modello di barare aumentando infinitamente l'incertezza (σ) per azzerare la loss.
def kendall_loss(task_loss, log_var):
    log_var = torch.clamp(log_var, min=-5.0, max=5.0) # Clip per stabilità numerica
    precision = torch.exp(-log_var)
    return precision * task_loss + log_var

# CALCOLO LOSS CON MASCHERE
# Qui gestiamo i "Dati Finti" (Padding Labels).
def compute_mtl_loss(outputs, batch, tox_weights, gamma):
    total_loss = torch.tensor(0.0, device=outputs['toxicity'].device)
    
    # TASK 1: TOSSICITÀ
    tox_mask = batch['task_mask_tox']
    if tox_mask.sum() > 0: # Se ci sono esempi tossici veri nel batch...
        loss = focal_loss(outputs['toxicity'][tox_mask], batch['tox_label'][tox_mask], gamma, tox_weights)
        total_loss += kendall_loss(loss, outputs['log_var_tox'])
        
    # TASK 2: EMOZIONI
    # Usiamo Binary Cross Entropy (BCE) perché è un problema Multi-Label (una frase può essere sia Triste che Arrabbiata)
    emo_mask = batch['task_mask_emo']
    if emo_mask.sum() > 0:
        loss = F.binary_cross_entropy_with_logits(outputs['emotion'][emo_mask], batch['emo_label'][emo_mask])
        total_loss += kendall_loss(loss, outputs['log_var_emo'])
        
    # TASK 3: SENTIMENT
    sent_mask = batch['task_mask_sent']
    if sent_mask.sum() > 0:
        loss = focal_loss(outputs['sentiment'][sent_mask], batch['sent_label'][sent_mask], gamma)
        total_loss += kendall_loss(loss, outputs['log_var_sent'])
        
    return total_loss

In [ ]:
# 5. DATASET & COLLATOR
# Gestione avanzata dei tensori.

class BaseDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_len):
        self.df = pd.read_csv(csv_path)
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.df)
    def encode(self, text):
        # TEXT PADDING: Tronca a 128 o riempie con 0 fino a 128.
        return self.tokenizer.encode_plus(
            str(text), max_length=self.max_len, padding='max_length', 
            truncation=True, return_tensors='pt'
        )

class ToxicityDataset(BaseDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(),
            'attention_mask': enc['attention_mask'].flatten(),
            # DATI REALI
            'tox_label': torch.tensor(int(row['label']), dtype=torch.long),
            # DATI FINTI (Label Padding) - verranno ignorati dalla mask
            'emo_label': torch.zeros(len(EMO_COLS)), 
            'sent_label': torch.tensor(-1, dtype=torch.long),
            # PASSAPORTO
            'task': 'toxicity'
        }

# (Emotion e Sentiment dataset omessi per brevità, stessa logica...)
# ...
class EmotionDataset(BaseDataset):
    def __init__(self, csv_path, tokenizer, max_len, emo_cols):
        super().__init__(csv_path, tokenizer, max_len)
        self.emo_cols = emo_cols
        if 'label_sum' in self.df.columns:
             self.df = self.df[self.df['label_sum'] > 0].reset_index(drop=True)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(),
            'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(-1, dtype=torch.long),
            'emo_label': torch.tensor([float(row[c]) for c in self.emo_cols]),
            'sent_label': torch.tensor(-1, dtype=torch.long),
            'task': 'emotion'
        }

class SentimentDataset(BaseDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(),
            'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(-1, dtype=torch.long),
            'emo_label': torch.zeros(len(EMO_COLS)),
            'sent_label': torch.tensor(int(row['label']), dtype=torch.long),
            'task': 'sentiment'
        }

def collate_fn(batch):
    # Questo è il 'vigile urbano' che smista il traffico nel batch
    tasks = [x['task'] for x in batch]
    return {
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
        'tox_label': torch.stack([x['tox_label'] for x in batch]),
        'emo_label': torch.stack([x['emo_label'] for x in batch]),
        'sent_label': torch.stack([x['sent_label'] for x in batch]),
        # CREAZIONE MASCHERE
        # Trasforma i passaporti stringa in array booleani per la GPU
        'task_mask_tox': torch.tensor([t == 'toxicity' for t in tasks], dtype=torch.bool), 
        'task_mask_emo': torch.tensor([t == 'emotion' for t in tasks], dtype=torch.bool),
        'task_mask_sent': torch.tensor([t == 'sentiment' for t in tasks], dtype=torch.bool)
    }


In [ ]:
# 6. LOADERS
tokenizer = BertTokenizer.from_pretrained(CONFIG['encoder'])

# ConcatDataset crea un flusso misto (Interleaved Dataset).
# Questo assicura che il modello non dimentichi un task mentre ne impara un altro.
tox_train = ToxicityDataset(f'{DATA_DIR}/toxicity_train.csv', tokenizer, CONFIG['max_length'])
emo_train = EmotionDataset(f'{DATA_DIR}/emotions_train.csv', tokenizer, CONFIG['max_length'], EMO_COLS)
sent_train = SentimentDataset(f'{DATA_DIR}/sentiment_train.csv', tokenizer, CONFIG['max_length'])
tox_val = ToxicityDataset(f'{DATA_DIR}/toxicity_val.csv', tokenizer, CONFIG['max_length'])

train_set = ConcatDataset([tox_train, emo_train, sent_train])
train_loader = DataLoader(train_set, batch_size=CONFIG['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader = DataLoader(tox_val, batch_size=CONFIG['batch_size'], shuffle=False, collate_fn=collate_fn)

tox_weights = torch.tensor([0.75, 1.5], device=device)

In [ ]:
# 7. TRAINING LOOP (The Engine)

def train_epoch(model, loader, optimizer, scheduler, config, tox_weights):
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    pbar = tqdm(loader, desc='Training')
    
    for step, batch in enumerate(pbar):
        # SPOSTAMENTO SU GPU
        for k, v in batch.items(): 
            if isinstance(v, torch.Tensor): batch[k] = v.to(device)
        
        # FORWARD
        outputs = model(batch['input_ids'], batch['attention_mask'])
        
        # LOSS Calculation
        loss = compute_mtl_loss(outputs, batch, tox_weights, config['focal_gamma'])
        
        # BACKWARD (Accumulato)
        (loss / config['gradient_accumulation']).backward()
        
        # OPTIMIZER STEP (ogni N batch)
        if (step + 1) % config['gradient_accumulation'] == 0:
            nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Evita exploding gradients
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item()
        
        # LOGGING SIGMA (Incertezza)
        if step % 50 == 0:
            σ_t = torch.exp(0.5 * model.log_var_tox).item()
            σ_e = torch.exp(0.5 * model.log_var_emo).item()
            pbar.set_postfix({'loss': loss.item(), 'σ_tox': f'{σ_t:.2f}', 'σ_emo': f'{σ_e:.2f}'})
    
    return total_loss / len(loader)

@torch.no_grad()
def validate(model, loader):
    model.eval()
    preds, labels = [], []
    for batch in tqdm(loader, desc='Validating', leave=False):
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        lbl = batch['tox_label'].to(device)
        out = model(ids, mask)
        preds.extend(out['toxicity'].argmax(1).cpu().numpy())
        labels.extend(lbl.cpu().numpy())
    
    # MACRO F1: La metrica più onesta per classi sbilanciate.
    # Calcola F1 per 'Non Tossico' e F1 per 'Tossico' separatamente e ne fa la media.
    return f1_score(labels, preds, average='macro')

In [ ]:
# 8. OPTIMIZER: Differential Learning Rates

model = AURA_CERBERUS(CONFIG).to(device)

optimizer = torch.optim.AdamW([
    # LAYER 1: BERT BACKBONE
    # LR molto basso (2e-5). Deve solo raffinare le sue conoscenze.
    {'params': model.bert.parameters(), 'lr': CONFIG['lr_bert']},
    
    # LAYER 2: HEADS E SIGMA
    # LR più alto (5e-5). Devono imparare velocemente da zero.
    {'params': model.toxicity_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.emotion_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.sentiment_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': [model.log_var_tox, model.log_var_emo, model.log_var_sent], 'lr': CONFIG['lr_heads']}
], weight_decay=CONFIG['weight_decay'])

total_steps = len(train_loader) * CONFIG['epochs'] // CONFIG['gradient_accumulation']
scheduler = OneCycleLR(optimizer,
    max_lr=[CONFIG['lr_bert']] + [CONFIG['lr_heads']]*4,
    total_steps=total_steps, pct_start=CONFIG['warmup_ratio'], anneal_strategy='cos')

In [ ]:
# 9. EXECUTION

best_val_f1 = 0.0
history = {'loss': [], 'val_f1': [], 'sigma_tox': [], 'sigma_emo': [], 'sigma_sent': []}

for epoch in range(1, CONFIG['epochs'] + 1):
    avg_loss = train_epoch(model, train_loader, optimizer, scheduler, CONFIG, tox_weights)
    val_f1 = validate(model, val_loader)
    
    # Monitoriamo le Incertezze (Sigma)
    # Se Sigma sale -> Il modello trova il task difficile.
    # Se Sigma scende -> Il modello sta imparando.
    σ_tox = torch.exp(0.5 * model.log_var_tox).item()
    σ_emo = torch.exp(0.5 * model.log_var_emo).item()
    σ_sent = torch.exp(0.5 * model.log_var_sent).item()
    
    history['loss'].append(avg_loss)
    history['val_f1'].append(val_f1)
    history['sigma_tox'].append(σ_tox)
    history['sigma_emo'].append(σ_emo)
    history['sigma_sent'].append(σ_sent)
    
    print(f'Epoch {epoch}: Loss={avg_loss:.4f}, Val F1={val_f1:.4f}')
    print(f'Uncertainties: Tox={σ_tox:.2f}, Emo={σ_emo:.2f}, Sent={σ_sent:.2f}')
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), 'aura_cerberus_best.pt')
        print('>>> NEW BEST MODEL <<<')

In [ ]:
# 10. VISUALIZZAZIONE
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))

# Loss
plt.subplot(1, 3, 1)
plt.plot(history['loss'], marker='o')
plt.title('Total Loss (Weighted)')
plt.grid()

# Validation F1
plt.subplot(1, 3, 2)
plt.plot(history['val_f1'], marker='s', color='orange')
plt.title('Toxicity F1 Score (Macro)')
plt.grid()

# Task Uncertainties
plt.subplot(1, 3, 3)
plt.plot(history['sigma_tox'], label='Tox')
plt.plot(history['sigma_emo'], label='Emo')
plt.plot(history['sigma_sent'], label='Sent')
plt.title('Homoscedastic Uncertainty (Sigma)')
plt.legend()
plt.grid()

plt.show()